In [1]:
import psycopg2
# import sqlalchemy as sq
import pandas as pd
import numpy as np
import mariadb
# import dask.dataframe as dd
import json
import os
import shutil
import subprocess
from pathlib import Path
import pyodbc

In [2]:
# conn_lims = pyodbc.connect("Driver={SQL Server};"
#                             "Server=192.168.5.18\CROPNUT;"
#                             "Database=cropnuts;"
#                             "uid=thomasTsuma;pwd=GR^KX$uRe9#JwLc6")

In [3]:
# _ = pd.read_sql("select TOP(1000) * from SampleResults where group_name='Soil Analysis' AND analysis_name LIKE '%Complete Soil Analysis%'",con=conn_lims)

C:\Users\tommy\AppData\Roaming\Python\Python38\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


In [2]:
# _.to_csv("sl.csv")

In [3]:
soil_df = pd.read_csv("input/soil_analysis_cleaned.csv")

In [4]:
len(soil_df)

3681157

In [5]:
soil_df['analysis_name'] = soil_df['analysis_name'].str.split(",")

In [6]:
soil_df = soil_df.explode('analysis_name', ignore_index=True)


In [7]:
soil_df.to_csv("output/soil_single_analysis.csv")

In [2]:
soil_df = pd.read_csv("output/soil_single_analysis.csv")

In [8]:
soil_df

,Unnamed: 0.1,Unnamed: 0,sample_code,batch_date,analysis_name,chemical_name,result,unit_name
0,0,0,CF045SA0784,2012-07-09 00:00:00.000,% Organic Matter (OM),iron,165.00,ppm
1,0,0,CF045SA0784,2012-07-09 00:00:00.000,Complete Soil Analysis,iron,165.00,ppm
2,1,1,CF045SA0784,2012-07-09 00:00:00.000,% Organic Matter (OM),cec,16.60,meq/100g
3,1,1,CF045SA0784,2012-07-09 00:00:00.000,Complete Soil Analysis,cec,16.60,meq/100g
4,2,2,CF045SA0784,2012-07-09 00:00:00.000,% Organic Matter (OM),aluminium,481.00,ppm
...,...,...,...,...,...,...,...,...
6017449,4038899,4038899,CI118SA1115,2024-04-10 00:00:00.000,Zn,organic_carbon,1.31,%
6017450,4038899,4038899,CI118SA1115,2024-04-10 00:00:00.000,Fe,organic_carbon,1.31,%
6017451,4038899,4038899,CI118SA1115,2024-04-10 00:00:00.000,Mn),organic_carbon,1.31,%
6017452,4038899,4038899,CI118SA1115,2024-04-10 00:00:00.000,Soil sulphate (calcium phosphate),organic_carbon,1.31,%


In [9]:
soil_df = soil_df.dropna(subset="result")

In [10]:
df_ = soil_df[['analysis_name','chemical_name','unit_name']].value_counts()
df_.to_csv("output/soil_unit_per_chemical.csv")

In [11]:
df_ = df_.reset_index()

In [18]:
analysis_chem_unit_df = pd.DataFrame()
for analysis in df_['analysis_name'].unique():
    print(analysis)
    chems_count_ = df_.loc[df_['analysis_name'] == analysis]
    duplicates_chem = (chems_count_[chems_count_.duplicated(subset='chemical_name')])
    temp_analysis_chem_unit_df = pd.DataFrame()
    for index,row in duplicates_chem.iterrows():
        chemical = row['chemical_name']
        chems_count_2 = chems_count_.loc[chems_count_['chemical_name'] == chemical ]
        unit = chems_count_2.loc[chems_count_2['count'] == max(chems_count_2['count'])]['unit_name'].values[0]
        _ = pd.DataFrame({"analysis_name":[analysis],"chemical_name":[chemical], "unit_name":[unit]})
        temp_analysis_chem_unit_df = pd.concat([temp_analysis_chem_unit_df, _])
    analysis_chem_unit_df = pd.concat([analysis_chem_unit_df, temp_analysis_chem_unit_df])
analysis_chem_unit_df.to_csv("output/soil_unit_per_chemical_decision.csv")

Complete Soil Analysis with Recommendations
Basic Soil Analysis with Recommendations
Complete Soil Analysis
Soil Texture Analysis
Complete Soil Analysis (M3)
AFSIS Std Wet Chemistry Soil Analysis
Soil Carbon (Walkley Black)
Complete Soil Analysis (data only)
Standard Wet Chem Soil Analysis  + Micros
Basic Soil Analysis
Spectral Soil Texture Analysis
Available Nitrogen in Soil
Sulphur Analysis
Soil Nitrogen (Kjeldahl)
Standard Soil (Olsen P) + Micros
Total Carbon & Nitrogen
% Organic Matter (OM)
% Soil Nitrogen (N)
Cropnuts Wet Chem IR Calibration Soil Analysis
Available Nitrogen
Exchangeable Acidity (Hp) Analysis
ATA Ethiopia Soil Analysis
Soil Organic Carbon and Total Nitrogen (IR)
Zn
Soil Micronutrients (EDTA Cu
Soil sulphate (calcium phosphate)
 Mn)
 Fe
KTDA Soil Analysis
Exchangeable Aluminium in Soil
Complete Soil Analysis + Exch. Acidity
Mehlich 3 extract analysis
Soil pH in Water
Complete Soil Analysis - Co & Si (IFDC)
Exchangeable Acidity (Hp) in Soil
Standard Wet Chem Soil Ana

In [19]:
analysis_chem_unit_df

,analysis_name,chemical_name,unit_name
0,Complete Soil Analysis with Recommendations,organic_carbon,%
0,Complete Soil Analysis with Recommendations,ec_salts,uS/cm
0,Complete Soil Analysis with Recommendations,boron,ppm
0,Complete Soil Analysis with Recommendations,c/n_ratio,%
0,Complete Soil Analysis with Recommendations,copper,ppm
...,...,...,...
0,Advanced Soil Health Analysis,est_available_water_holding_capacity,%
0,Chromium in Soil,chromium,mg/kg
0,Morgan P,organic_carbon,ppm
0,Standard Soil Health Analysis (Alkaline),organic_carbon,ppm


In [20]:
analysis_chem_unit_df

,analysis_name,chemical_name,unit_name
0,Complete Soil Analysis with Recommendations,organic_carbon,%
0,Complete Soil Analysis with Recommendations,ec_salts,uS/cm
0,Complete Soil Analysis with Recommendations,boron,ppm
0,Complete Soil Analysis with Recommendations,c/n_ratio,%
0,Complete Soil Analysis with Recommendations,copper,ppm
...,...,...,...
0,Advanced Soil Health Analysis,est_available_water_holding_capacity,%
0,Chromium in Soil,chromium,mg/kg
0,Morgan P,organic_carbon,ppm
0,Standard Soil Health Analysis (Alkaline),organic_carbon,ppm


In [23]:
df = pd.DataFrame()
for index, row in analysis_chem_unit_df.iterrows():
    analysis = row['analysis_name']
    chemical = row['chemical_name']
    unit = row['unit_name']
    temp_df = soil_df.loc[(soil_df['analysis_name'] == analysis) & (soil_df['chemical_name'] == chemical) & (soil_df['unit_name'] == unit)]
    df = pd.concat([df, temp_df])

In [24]:
df

,Unnamed: 0.1,Unnamed: 0,sample_code,batch_date,analysis_name,chemical_name,result,unit_name
5708,3457,3457,FA611-161SA0001,2019-04-29 00:00:00.000,Complete Soil Analysis with Recommendations,organic_carbon,1.30,%
5858,3532,3532,FA807-384SA0001,2019-10-30 00:00:00.000,Complete Soil Analysis with Recommendations,organic_carbon,3.37,%
7053,4311,4311,CC004SA0484,2019-05-06 00:00:00.000,Complete Soil Analysis with Recommendations,organic_carbon,1.68,%
8124,5056,5056,CC004SA0485,2019-05-06 00:00:00.000,Complete Soil Analysis with Recommendations,organic_carbon,2.23,%
13057,7262,7262,FA807-384SA0002,2019-10-30 00:00:00.000,Complete Soil Analysis with Recommendations,organic_carbon,2.81,%
...,...,...,...,...,...,...,...,...
5423691,3703954,3703954,CA266SA2510,2024-06-13 16:08:00.000,Standard Soil Health Analysis (Alkaline),organic_carbon,0.85,ppm
5423793,3704000,3704000,CA266SA2511,2024-06-13 16:08:00.000,Standard Soil Health Analysis (Alkaline),organic_carbon,0.53,ppm
4587318,3110973,3110973,CW153SA0001,2021-08-20 15:30:00.000,Cobalt in Soil,molybdenum,3.96,mg/kg
4587321,3110974,3110974,CW153SA0001,2021-08-20 15:30:00.000,Cobalt in Soil,molybdenum,3.96,mg/kg


In [25]:
df.to_csv("output/soil_analysis_cleaned.csv",chunksize=1000)

In [26]:
os.getcwd()

'D:\\Cropnuts\\DSML158\\SoilAnalysis'